<a href="https://www.kaggle.com/code/imenbakir/google-stock-price-prediction?scriptVersionId=128674957" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LSTM
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
from tensorflow.keras import layers

In [29]:
df_train = pd.read_csv('/kaggle/input/google-stock-price/Google_Stock_Price_Train.csv')
df_train.head(20)

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"
5,1/10/2012,313.70,315.72,307.30,621.43,"8,824,000"
6,1/11/2012,310.59,313.52,309.40,624.25,"4,817,800"
7,1/12/2012,314.43,315.26,312.08,627.92,"3,764,400"
8,1/13/2012,311.96,312.30,309.37,623.28,"4,631,800"
9,1/17/2012,314.81,314.81,311.67,626.86,"3,832,800"


In [13]:
df_test = pd.read_csv('/kaggle/input/google-stock-price/Google_Stock_Price_Test.csv')
df_test.head(15)

,Date,Open,High,Low,Close,Volume
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300"
1,1/4/2017,788.36,791.34,783.16,786.90,"1,073,000"
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200"
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200"
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400"
5,1/10/2017,807.86,809.13,803.51,804.79,"1,176,800"
6,1/11/2017,805.00,808.15,801.37,807.91,"1,065,900"
7,1/12/2017,807.14,807.39,799.17,806.36,"1,353,100"
8,1/13/2017,807.48,811.22,806.69,807.88,"1,099,200"
9,1/17/2017,807.08,807.14,800.37,804.61,"1,362,100"


**selecting features**

In [31]:
df_train = df_train[['Open', 'Close']]

In [32]:
df_train

,Open,Close
0,325.25,663.59
1,331.27,666.45
2,329.83,657.21
3,328.34,648.24
4,322.04,620.76
...,...,...
1253,790.90,789.91
1254,790.68,791.55
1255,793.70,785.05
1256,783.33,782.79


**feature scaling**

In [34]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1258 non-null   float64
 1   Close   1258 non-null   object 
dtypes: float64(1), object(1)
memory usage: 19.8+ KB


In [37]:
df_train['Close'] = df_train['Close'].str.replace(',', '').astype(float)

In [38]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1258 non-null   float64
 1   Close   1258 non-null   float64
dtypes: float64(2)
memory usage: 19.8 KB


In [44]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range =(0,1))
df_train_scaled = sc.fit_transform(df_train)

**creating data structure with 60 timesteps and 1 output**

In [45]:
X_train = []
y_train = []
for i in range(60,1258):
    X_train.append(df_train_scaled[i-60:i, 0])
    y_train.append(df_train_scaled[i,0])

# transform to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

In [46]:
#reshaping 
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

voir documentation keras 

In [47]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [48]:
#initializing RNN
regressor = Sequential()

In [51]:
#adding first LSTM layer and some dropout regularization
regressor.add(LSTM(units=50 , return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0,2))

In [52]:
#adding second LSTM layer and some dropout regularization
regressor.add(LSTM(units=50 , return_sequences=True))
regressor.add(Dropout(0,2))

In [53]:
#adding thirs LSTM layer and some dropout regularization
regressor.add(LSTM(units=50 , return_sequences=True))
regressor.add(Dropout(0,2))

In [54]:
#adding fourth LSTM layer and some dropout regularization
regressor.add(LSTM(units=50))
regressor.add(Dropout(0,2))

In [55]:
#adding output layer
regressor.add(Dense(units=1))

In [56]:
#compiling the RNN with rmsprop
regressor.compile(optimizer='rmsprop', loss='mean_squared_error')

In [57]:
#fitting the model to training set
regressor.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
38/38 [==============================] - 12s 108ms/step - loss: 0.0332
Epoch 2/100
38/38 [==============================] - 4s 109ms/step - loss: 0.0094
Epoch 3/100
38/38 [==============================] - 4s 106ms/step - loss: 0.0085
Epoch 4/100
38/38 [==============================] - 4s 106ms/step - loss: 0.0073
Epoch 5/100
38/38 [==============================] - 4s 107ms/step - loss: 0.0059
Epoch 6/100
38/38 [==============================] - 4s 106ms/step - loss: 0.0059
Epoch 7/100
38/38 [==============================] - 4s 107ms/step - loss: 0.0047
Epoch 8/100
38/38 [==============================] - 4s 109ms/step - loss: 0.0051
Epoch 9/100
38/38 [==============================] - 4s 112ms/step - loss: 0.0043
Epoch 10/100
38/38 [==============================] - 4s 107ms/step - loss: 0.0043
Epoch 11/100
38/38 [==============================] - 4s 108ms/step - loss: 0.0035
Epoch 12/100
38/38 [==============================] - 4s 107ms/step - loss: 0.0039
Epoch 13/100

**Prediction**

In [67]:
df_test = pd.read_csv('/kaggle/input/google-stock-price/Google_Stock_Price_Test.csv')
df_test.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300"
1,1/4/2017,788.36,791.34,783.16,786.90,"1,073,000"
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200"
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200"
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400"


In [77]:
real_stock_price = df_test[['Open', 'Close']].values

In [85]:
# Getting the predicted stock price of 2017 (concaténer au niv des lignes)
dataset_total = pd.concat((df_train[['Open', 'Close']], df_test[['Open', 'Close']]), axis = 0)
#à transformer en array (values)
inputs = dataset_total[len(dataset_total) - len(df_test) - 60:].values

In [87]:
#changer le format de l'objet (essuyer sans la ligne suivante)
inputs = sc.transform(inputs)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"


In [90]:
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, :])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

predicted_stock_price = regressor.predict(X_test)

predicted_stock_price = sc.inverse_transform(predicted_stock_price)

InvalidArgumentError: Graph execution error:

Specified a list with shape [?,1] from a tensor with shape [20,2]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_predict_function_26385]

In [ ]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()

In [ ]:
loss = model_history.history['loss']
validation_loss = model_history.history['val_loss']



plt.figure(figsize=(17, 7));
plt.plot(range(100), loss, label='Training Loss');
plt.plot(range(100), validation_loss, label='Validation Loss');
plt.legend(loc='upper left');
plt.title('Loss : Training Vs Validation ');

**real vs predicted google stock price**

In [7]:
df = pd.concat([df_test,df_train])
df.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300"
1,1/4/2017,788.36,791.34,783.16,786.9,"1,073,000"
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200"
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200"
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400"


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1278 entries, 0 to 1257
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1278 non-null   object 
 1   Open    1278 non-null   float64
 2   High    1278 non-null   float64
 3   Low     1278 non-null   float64
 4   Close   1278 non-null   object 
 5   Volume  1278 non-null   object 
dtypes: float64(3), object(3)
memory usage: 69.9+ KB


In [9]:
df.describe()

,Open,High,Low
count,1278.000000,1278.000000,1278.000000
mean,537.994906,542.168897,533.278803
std,154.508365,155.582501,153.174011
min,279.120000,281.210000,277.220000
25%,406.037500,408.230000,403.335000
50%,538.395000,542.330000,534.355000
75%,668.862500,677.705000,662.190000
max,837.810000,841.950000,827.010000


In [10]:
df.shape

(1278, 6)

In [11]:
df_open = df.loc[:,["Open"]]